In [15]:
%matplotlib inline

In [19]:
%run prod_scripts/get_latest_event_round.ipynb

In [26]:
import os

In [27]:
from datetime import datetime

year = datetime.today().year

os.listdir('timing_data')

['2021_22_FP1_laps.csv',
 '2021_22_Q_laps.csv',
 '2021_22_Q_telem_fastest.csv',
 '2021_22_R_laps.csv',
 '2022_1_FP1_laps.csv',
 '2022_1_FP1_telem_all.csv',
 '2022_1_fp2_laps.csv',
 '2022_1_q_laps.csv',
 '2022_1_q_telem_fastest.csv',
 '2022_1_R_laps.csv',
 '2022_1_R_telem_all.csv',
 '2022_1_R_telem_fastest.csv',
 '2022_2_FP1_laps.csv',
 '2022_2_FP1_telem_all.csv',
 '2022_2_FP2_laps.csv',
 '2022_2_FP2_telem_all.csv',
 '2022_2_Q_laps.csv',
 '2022_2_Q_telem_all.csv',
 '2022_2_Q_telem_fastest.csv',
 '2022_2_R_laps.csv',
 '2022_2_R_telem_all.csv',
 '2022_3_FP2_laps.csv',
 '2022_3_FP2_telem_all.csv',
 '2022_7_R_laps.csv',
 '2022_7_R_telem_all.csv',
 '2022_7_R_telem_fastest.csv']

In [28]:
import re

regex = re.compile(str(year) + "_" + str(round_number) + '_R_telem_all')

In [29]:
# Get path of files that match the regex
for root, dirs, files in os.walk('timing_data'):
  l = [os.path.join(root,x) for x in files if regex.match(x)]
  if l: print(l)

['timing_data\\2022_7_R_telem_all.csv']


In [30]:
import pandas as pd
import numpy as np

In [31]:
race_path = l[0]

In [32]:
df = pd.read_csv(race_path)

In [33]:
df.head()

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Distance,Driver,Team,Lap
0,2022-05-29 14:36:38.413,11904,257,6,100,True,1,car,0 days 00:00:00.165000,0 days 02:35:38.392000,11.779167,MAG,Haas F1 Team,17
1,2022-05-29 14:36:38.613,11454,256,6,2,True,1,car,0 days 00:00:00.365000,0 days 02:35:38.592000,26.001389,MAG,Haas F1 Team,17
2,2022-05-29 14:36:38.853,10614,240,6,0,True,1,car,0 days 00:00:00.605000,0 days 02:35:38.832000,42.001389,MAG,Haas F1 Team,17
3,2022-05-29 14:36:39.133,9688,225,6,0,True,1,car,0 days 00:00:00.885000,0 days 02:35:39.112000,59.501389,MAG,Haas F1 Team,17
4,2022-05-29 14:36:39.573,9829,193,6,0,True,1,car,0 days 00:00:01.325000,0 days 02:35:39.552000,83.090278,MAG,Haas F1 Team,17


In [34]:
import plotly.express as px

In [35]:
df.columns

Index(['Date', 'RPM', 'Speed', 'nGear', 'Throttle', 'Brake', 'DRS', 'Source',
       'Time', 'SessionTime', 'Distance', 'Driver', 'Team', 'Lap'],
      dtype='object')

In [36]:
regex = re.compile(str(year) + "_" + str(round_number) + '_R_laps')

# Get path of files that match the regex
for root, dirs, files in os.walk('timing_data'):
  l = [os.path.join(root,x) for x in files if regex.match(x)]
  if l: print(l)

laps_path = l[0]        

df_laps = pd.read_csv(laps_path)

['timing_data\\2022_7_R_laps.csv']


In [37]:
df_laps.columns

Index(['Time', 'DriverNumber', 'LapTime', 'LapNumber', 'Stint', 'PitOutTime',
       'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
       'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
       'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
       'Compound', 'TyreLife', 'FreshTyre', 'LapStartTime', 'Team', 'Driver',
       'TrackStatus', 'IsAccurate'],
      dtype='object')

In [49]:
# convert LapTime from string to float
ms_regex = re.compile('\.[0-9]+$')
re.match(df_laps['LapTime'])

TypeError: match() missing 1 required positional argument: 'string'

In [52]:
df_laps['ms'] = df_laps['LapTime'].str.extract(r'(\.[0-9]+$)')

In [54]:
df_laps['secs'] = df_laps['LapTime'].str.extract(r'([0-9]+)\.')

In [56]:
df_laps['mins'] = df_laps['LapTime'].str.extract(r'\:([0-9]+)\:')

In [68]:
df_laps['LapTimeinSeconds'] = df_laps['mins'].astype(float)*60 + df_laps['secs'].astype(float) + df_laps['ms'].astype(float)

In [73]:
# group avg. laptimes for plotting
df_laps_plot = df_laps.groupby(['LapNumber', 'Compound']).mean()

df_laps_plot.head()

DriverNumber     Stint     SpeedI1     SpeedI2  \
LapNumber Compound                                                       
1         WET              24.700000  2.850000  110.950000  114.150000   
2         INTERMEDIATE     24.600000  2.800000   94.600000  120.800000   
          WET              24.733333  3.000000   98.400000  120.466667   
3         INTERMEDIATE     21.500000  3.166667  131.833333  130.666667   
          WET              26.071429  3.071429  126.642857  130.000000   

                           SpeedFL     SpeedST  TyreLife  TrackStatus  \
LapNumber Compound                                                      
1         WET           163.777778  165.850000  1.000000          4.2   
2         INTERMEDIATE  211.500000  155.200000       NaN          4.0   
          WET           206.846154  147.533333  1.933333          4.0   
3         INTERMEDIATE  229.000000  236.833333  1.000000          1.0   
          WET           227.071429  222.785714  2.857143          1.0   

                        IsAccurate  LapTimeinSeconds  
LapNumber Compound                                    
1         WET                  0.0               NaN  
2         INTERMEDIATE         0.0        134.787750  
          WET                  0.0        136.156400  
3         INTERMEDIATE         0.0        106.107333  
          WET                  0.0        107.422143

In [97]:
df_laps_plot.columns

df_laps_plot.index

df_laps_plot = df_laps_plot.reset_index(drop = False)

In [116]:
fig = px.line(
    data_frame=df_laps_plot,
    x = 'LapNumber',
    y = 'LapTimeinSeconds',
    color = 'Compound',
    title = 'Lap Times (seconds) by Tire Compound',
    labels = {
        'LapNumber' : 'Lap Number',
        'LapTimeinSeconds' : 'Lap Time (s)'
    },
    template = 'plotly_white',
    color_discrete_sequence = px.colors.qualitative.Safe
             )

fig.show()